In [1]:
import numpy as np
import json
from pathlib import Path
import shutil

In [3]:
base_dir = Path("./")
for scenario_dir in base_dir.iterdir():
    if not scenario_dir.is_dir():
        continue

    for data_file in scenario_dir.glob("onlyq_*.json"):
        
        # # copy data_file 
        data_file_backup = data_file.with_suffix(".json.bak")
        # shutil.copy(data_file, data_file_backup)

        with open(data_file_backup, "r", encoding="utf-8") as f:
            datas = json.loads(f.read())

        with open(data_file, "w", encoding="utf-8") as f:
            new_datas = []
            for data in datas:
                input, response = data["Input"], data["Response"]
                formalized_input, input_semantic_parsing, strategy = \
                    response["Formalized Input"], response["Input Semantic Parsing"], response["Strategy"]
                
                isp_flatten = {}
                for k, v in input_semantic_parsing.items():
                    isp_flatten.update(v)
                
                isp_flatten = [f"'{k}':'{v}'" for k, v in isp_flatten.items()]

                # for k, v in isp_flatten.items():
                #     try:
                #         # search for the key in the formalized input
                #         idx = formalized_input.index(k)
                #         formalized_input[idx] = f"{k}({v})"
                #     except ValueError:
                #         print(f)

                steps = strategy["Step"]
                steps = '\n'.join([f"{i+1}. {step}" for i, step in enumerate(steps)])

                cot_text = \
f"""
사용자는 '{formalized_input}'라고 입력함. {', '.join(isp_flatten)}에 해당함.
사용자의 의도를 추측하면, 유저는 '{', '.join(strategy['Expected Outputs'])}' 와 비슷한 답변을 기대할것.
퀄리티 있는 대답을 위해 나는 다음 단계를 수행.
{steps}
"""
                # print(cot_text)

                new_response = {
                    '생각': cot_text,
                    '지시': response['Instruction Set'],
                }

                new_data = {
                    'Input': input,
                    'Response': new_response,
                }
                new_datas.append(new_data)
            f.write(json.dumps(new_datas, ensure_ascii=False, indent=4))

Key '가장 최근(1주일)' not found in the formalized input '옆반(01_IB7)반의 가장 최근 실내온도와 설정온도를 알려줘.'
Key '시간' not found in the formalized input '현재(2021-08-05 14:30:00)재 몇 시인지 알려줘.'
Key '8일 전' not found in the formalized input '8일전 우리반의 일평균 설정온도를 알려줘.'
Key '현재' not found in the formalized input '1층의 평균 실내온도를 알려줘.'
Key '에어컨 전원' not found in the formalized input '현재(2021-08-05 14:30:00)재 에어컨이 꺼져있는 방들을 알려줘.'
Key '설정온도' not found in the formalized input '현재(2021-08-05 14:30:00)재 에어컨 전원(oper)어컨 전원이 켜져있으며 설정 온도가 제일 낮은 방 3개 알려줘.'
Key '어제 저녁' not found in the formalized input '옆반 어제 아침(2021-08-04 06:00:00 ~ 2021-08-04 09:00:00)제 아침과 저녁의 설정온도 평균의 차이 알려줘.'
Key '현재' not found in the formalized input '3층의 평균 실내온도를 알려줘.'
Key '오늘 저녁' not found in the formalized input '우리반 오늘 아침(2021-08-05 06:00:00 ~ 2021-08-05 09:00:00)늘 아침과 저녁의 실내온도 차이 알려줘.'
Key '실내온도' not found in the formalized input '지난 3일(2021-08-02 00:00:00 ~ 2021-08-04 11:59:59)난 3일 동안 우리반(01_IB5)리반 실내 온도 평균 값을 알려줘.'
Key '가장 최근(1주일)' not found in the for